In [1]:
import dask.dataframe as ddf
import pandas as pd
import json
import os
import numpy as np
import pickle
from glob import glob

In [2]:
path = '../../tcm-columns-add-main/'

In [3]:
df = pd.read_parquet(path + 'page.title')

In [4]:
selected_editors = pd.read_parquet('../../intermediate-result/TCM/editors-with-sig-contrib-at-least-10').index

In [6]:
editor_df = pd.read_parquet(path + 'contributor.username')

In [9]:
df = editor_df.join(df)

In [13]:
df = df[df['contributor.username'].isin(selected_editors)]

In [19]:
df = df.groupby(['contributor.username','page.title']).agg({'page.title':'count'}).rename(columns={'page.title':'count'}).reset_index()

In [21]:
def is_talk(title):
    return title.startswith('Talk:')

In [24]:
df['is_talk'] = df['page.title'].apply(is_talk)

In [26]:
article = df[~df['is_talk']]

In [29]:
article = article.pivot(index='contributor.username',columns='page.title',values='count')

In [32]:
article = article.fillna(0)

In [38]:
article.to_csv('../../result/TCM/TCM-editor-article-revision-matrix.csv', sep="\t",encoding='utf-16')

In [39]:
talks = df[df['is_talk']]

In [41]:
talks = talks.pivot(index='contributor.username',columns='page.title',values='count')

In [42]:
talks = talks.fillna(0)

In [44]:
talks.to_csv('../../result/TCM/TCM-editor-article-revision-matrix-talk-pages.csv', sep="\t",encoding='utf-16')